# Imports

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install pillow

In [ ]:
import subprocess
import tkinter as tk
import turtle

import numpy as np
from scipy.stats import linregress
from PIL import ImageGrab

# Draw image

In [ ]:
def dump_gui(window, filename = "gui_image_grabbed.png"):
    """
    takes a png screenshot of a tkinter window, and saves it on in cwd
    """
    print('...dumping gui window to png')

    x0 = window.winfo_rootx()
    y0 = window.winfo_rooty()
    x1 = x0 + window.winfo_width()
    y1 = y0 + window.winfo_height()
    ImageGrab.grab().crop((x0, y0, x1, y1)).save(filename)

## Sierpinski Triangle

In [ ]:
def draw_sierpinski(turtle, length, depth):
    if depth == 0:
        for i in range(0, 3):
            turtle.fd(length)
            turtle.left(120)
    else:
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.fd(length / 2)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.bk(length / 2)
        turtle.left(60)
        turtle.fd(length / 2)
        turtle.right(60)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.left(60)
        turtle.bk(length / 2)
        turtle.right(60)

## Run

In [ ]:
root = tk.Tk()
canvas = tk.Canvas(root, width=500, height=500)
canvas.pack()

t = turtle.RawTurtle(canvas)

t.penup()
t.goto(-200, -175)
t.pendown()
draw_sierpinski(t, 400, 3)
t.hideturtle()

dump_gui(root, "Triangle.png")

root.mainloop()

# Run C++ executable

In [ ]:
runTimes = 10
filename = "Triangle.png"

subprocess.call(args=["./build/FractalDimensionComputation", "-f", f"{filename}", "-t", f"{runTimes}"], stdin=None, stdout=None, stderr=None, shell=False)

# Calculate statistics

In [ ]:
times = []
with open("results_time.txt", "r") as fin:
    for _ in range(runTimes):
        times.append(float(fin.readline()[:-1]))

for i in range(runTimes):
    sizes = [] # x = s
    boxes = [] # y = n(s)
    with open(f"results_{i}.txt", "r") as fin:
        line = fin.readline()[:-1].split(' ')
        while line != [""]:
            sizes.append(np.log(1 / int(line[0])))
            boxes.append(np.log(int(line[1])))
            
            line = fin.readline()[:-1].split(' ')
    
    print("----------------------------------")
    print(f"Run {i}")
    print(f"Dimension = {linregress(sizes, boxes).slope}")
    print(f"Time spent: {times[i]}")
print(f"Time spent mean: {np.mean(times)}")
print(f"Time spent standard deviation: {np.std(times)}")